# first implementation, the goal will be to adapt so it fits to a river implementation.

Notes sur la lecture de l'article :
- ce modèle complète d'autres modèles (ALMID -> CALMID)

questions sur le papier (trouvable en allant voir les autres papiers)
- utilisation du ADWIN (de ce que je vois, soit ils en font un soit ils font des updates avec 0 si bonne pred ou 1 si mauvaise pred) <-- ce que je fais pour l'instant
- training en utilisant les weights --> comment ça marche ? réponse pour l'instant : tirer une poisson et entrainer x fois selon le résultat
- quelle stratégie pour trouver le worst_clf ? jsp

river implementation ideas
- class inheritance for EnsembleClassifier

In [5]:
import numpy as np
from river import drift
from river.tree import HoeffdingTreeClassifier
from typing import List
from river import datasets
from tqdm import tqdm
from river import base

In [12]:
class LIFO:
    """simple Last In First Out queue
    """
    def __init__(self, max_size: int):
        self.max_size = max_size
        self.queue = []
        
        
    def __len__(self):
        return len(self.queue)
    

    def add(self, item):
        self.queue.append(item)
        if len(self.queue) > self.max_size:
            self.queue.pop(0)
            
            
    def count(self, x):
        return self.queue.count(x)

In [13]:
class CALMID(base.Classifier, base.Ensemble):
    
    def __init__(
        self,
        C: int,
        theta: float,
        s,
        epsilon: float,
        b: float,
        sizelab: int,
        classifiers: list[base.Classifier]
        ) -> None:
        """CALMID class constructor"""
        # is it normal that we have to pass C ?
        if b <= epsilon:
            raise ValueError("b must be greater than epsilon")
        if not 0 <= epsilon <= 1:
            raise ValueError("epsilon must be between 0 and 1")
    
        self.C = C
        self.theta = theta
        self.s = s
        self.epsilon = epsilon
        self.b = b
        self.sizelab = sizelab
    
        self.t=0
        self.l=0
        self.sizesam = int(sizelab*epsilon/C) # should be ceiled or floored ? --> they ceil it in the paper
        
        self.L = LIFO(max_size=self.sizelab)
        self.P = [LIFO(max_size=self.sizesam) for _ in range(self.C)]
        self.M = np.full((self.C, self.C), self.theta)
        
        self.classifiers = classifiers
        self.adwin = drift.ADWIN()
        
    def learn_one_ensemble(self, x, y, weight, decay_factor):
        decayed_weight = decay_factor * weight
        w = np.random.poisson(decayed_weight)
        for _ in range(w):
            [clf.learn_one(x, y) for clf in self.classifiers]
        
        
    def learn_one(self, x, y): 
        self.t += 1
        labelling = False
        zeta = np.random.uniform(0, 1)
        
        if self.t<self.sizelab or zeta<self.epsilon:
            self.L.add(y)
            labelling = True
        elif self.uncertainty_selective_strategy(x, y) and self.l/self.t < self.b:
            self.L.add(None)
            labelling = True
        else:
            self.L.add(None)
        
        if labelling:
            self.l += 1
            self.adwin.update(int(self.predict_one(x) == y))
            w = self.compute_weight(x, y)
            self.learn_one_ensemble(x, y, w, self.t)
            self.P[y].add((x, y, w, self.t))
            
            if self.adwin.drift_detected:
                print(self.t, "drift detected")
                new_base_clf = BaseClassifier()
                self.initalize_base_classifiers(new_base_clf)
                self.replace_worst_clf(new_base_clf, x, y)
    
    
        
    def predict_one(self, x):
        predictions = [clf.predict_one(x) for clf in self.classifiers]
        return max(set(predictions), key = predictions.count)
    
    
    def predict_proba_one(self, x):
        predictions = [clf.predict_proba_one(x) for clf in self.classifiers]
        try:
            return np.mean(predictions, axis=0).tolist()
        except:
            return [1] + [0] * (self.C - 1) 
                
                
    def uncertainty_selective_strategy(self, x, y) -> bool:
        labelling = False
        margin, yc1, yc2 = self.compute_probability_margin_and_top_classes(x)
        if margin <= self.M[yc1, yc2]:
            labelling = True
            imb_y = self.compute_imbalance(y)
            if y == yc1:
                self.M[yc1, yc2] *= (1-self.s)
                if imb_y > 0.5:
                    self.M[yc1, yc2] *= (1-self.s)
            elif y == yc2 and imb_y > 0.5:
                self.M[yc1, yc2] *= (1-self.s)
        else:
            sampbudget = self.b - self.l/self.t
            q = margin - self.M[yc1, yc2]
            sampbudget = sampbudget / (sampbudget + q)
            zeta = np.random.uniform(0, 1)
            if zeta < sampbudget:
                labelling = True
            if labelling and y == yc2:
                self.M[yc1, yc2] = max([self.theta, self.M[yc1, yc2]*(1+self.s)])
        return labelling
    
    
    def compute_sample_difficulty(self, x, y) -> float:
        margin, yc1, yc2 = self.compute_probability_margin_and_top_classes(x)
        if yc1 == y:
            tf, s = 1, 0
        elif yc2 == y:
            tf, s = -1, 1
        else:
            tf, s = -1, 0
        return (1 - tf * margin) * np.exp(1 - tf - s)
    
    
    def compute_weight(self, x, y) -> float:
        imb_y = max(1, self.compute_imbalance(y)) # to avoid division by 0
        return np.log(1 + self.compute_sample_difficulty(x, y) + 1/imb_y)
    
    
    def compute_imbalance(self, y) -> float:
        return self.L.count(y) / ((len(self.L) - self.L.count(None)) /self.C)
    
    
    def compute_probability_margin_and_top_classes(self, x) -> float: 
        predictive_probas = self.predict_proba_one(x)
        indexed_elements = list(enumerate(predictive_probas))
        sorted_elements = sorted(indexed_elements, key=lambda x: x[1], reverse=True)
        yc1, p_yc1 = sorted_elements[0]
        yc2, p_yc2 = sorted_elements[1]
        return p_yc1 - p_yc2, yc1, yc2
    
    
    def initalize_base_classifiers(self, base_clf : base.Classifier):
        sample_sequence = []
        for i in range(self.C):
            for j in range(self.sizesam):
                sample_sequence.add(self.P[i][j])
        sorted_sample_sequence = sorted(sample_sequence, key=lambda x: x[3]) # sort by timestamp
        for sample_x, sample_y, sample_weight, sample_arriving_time in sorted_sample_sequence:
            decayed_weight = self.compute_decay_factor(sample_arriving_time) * sample_weight
            w = np.random.poisson(decayed_weight)
            for _ in range(w):
                base_clf.learn_one(sample_x, sample_y)
            
            
    def compute_decay_factor(self, arriving_time):
        return np.exp(-(self.t - arriving_time)/self.sizelab)
    
        
    def replace_worst_clf(self, new_base_clf, x, y):
        worst_clf = self.get_worst_clf_index(x, y)
        if worst_clf == len(self.classifiers):
            self.classifiers.append(new_base_clf)
        else:
            self.classifiers[worst_clf] = new_base_clf
            
    
    def get_worst_clf_index(self, x, y):
        # TODO: how do we find which one is "worst"? last example ? we keep a trace ?
        # method if "last example"
        if len(self.classifiers) < self.D:
            return len(self.classifiers)
        else :
            return np.argmin([clf.predict_proba_one(x)[y] for clf in self.classifiers])
        

In [14]:
dataset = datasets.Insects(variant="incremental_imbalanced")
stream = iter(dataset)

In [15]:
calmid = CALMID(
    C=6,
    theta=0.4,
    s=0.1,
    epsilon=0.1,
    b=0.4,
    sizelab=600,
    classifiers = [HoeffdingTreeClassifier() for _ in range(10)]
)

label_to_id = {}
res = []
for i, (x, y) in tqdm(enumerate(stream)):
    if y not in label_to_id:
        label_to_id[y] = len(label_to_id)
    y = label_to_id[y]
    y_hat = calmid.predict_one(x)
    res.append(y_hat == y)
    calmid.learn_one(x, y)


0it [00:00, ?it/s]

0it [06:56, ?it/s]


KeyboardInterrupt: 

In [1]:
import pandas as pd
pd.Series(res).rolling(100).mean().plot()

NameError: name 'res' is not defined

In [17]:
pd.Series(res).rolling(100)

Rolling [window=100,center=False,axis=0,method=single]
